In [40]:
import mysql.connector

conn = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "42krScm9AYbnucK.root",
  password = "aBNY8oPEWPSEo2YM",
  database = "vector_db",
  ssl_ca = "isrgrootx1.pem",
  ssl_verify_cert = True,
  ssl_verify_identity = True
)

In [41]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("BAAI/bge-m3")
text = "I love Machine Learning!"
embedding = embedder.encode(text).tolist()

Loading weights: 100%|██████████| 391/391 [00:00<00:00, 764.67it/s, Materializing param=pooler.dense.weight]                               


In [42]:
len(embedding)

1024

In [43]:
documents = [
    "คณะวิทยาศาสตร์ มหาวิทยาลัยสงขลานครินทร์ วิทยาเขตหาดใหญ่ จัดตั้งขึ้นในปี พ.ศ. 2510 ซึ่งเป็นปีเดียวกับการก่อตั้งมหาวิทยาลัยสงขลานครินทร์",
    "คณะวิทยาศาสตร์ ประกอบด้วย 4 สาขา ได้แก่ วิทยาศาสตร์กายภาพ วิทยาศาสตร์ชีวภาพ วิทยาศาสตร์การคำนวณ และวิทยาศาสตร์สุขภาพและวิทยาศาสตร์ประยุกต์",
    "จัดการเรียนการสอนออกเป็นหลักสูตรปริญญาตรี 13 หลักสูตร และระดับบัณฑิตศึกษา 28 หลักสูตร คือปริญญาโท 17 หลักสูตร (มีหลักสูตรนานาชาติ 5 หลักสูตร) และปริญญาเอก 11 หลักสูตร (มีหลักสูตรนานาชาติ 6 หลักสูตร)",
    "วิสัยทัศน์ คือ คณะวิทยาศาสตร์เพื่อการพัฒนาที่ยั่งยืน",
    "คณบดี คือ ศาสตราจารย์ ดร.อัญชนา ประเทพ"
]

In [44]:
import json

def add_document(conn, text):
    cur = conn.cursor()
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (document, embedding) VALUES (%s , %s)", (text, json.dumps(embedding)))
    conn.commit()
    cur.close()

for doc in documents:
    add_document(conn, doc)

In [45]:
def query_tidb(conn, query_text, k=5):
    query_embedding = json.dumps(embedder.encode(query_text).tolist())
    
    cur = conn.cursor()

    sql_query = f"""
        SELECT document, vec_cosine_distance(embedding, '{query_embedding}') AS distance
        FROM documents
        ORDER BY distance
        LIMIT {k}
    """

    cur.execute(sql_query)
    results = cur.fetchall()
    cur.close()

    return [(content, distance) for content, distance in results]

results = query_tidb(conn, "คณะวิทย์มีสาขาอะไรบ้าง")
results

[('คณะวิทยาศาสตร์ ประกอบด้วย 4 สาขา ได้แก่ วิทยาศาสตร์กายภาพ วิทยาศาสตร์ชีวภาพ วิทยาศาสตร์การคำนวณ และวิทยาศาสตร์สุขภาพและวิทยาศาสตร์ประยุกต์',
  0.3143350635373634),
 ('วิสัยทัศน์ คือ คณะวิทยาศาสตร์เพื่อการพัฒนาที่ยั่งยืน', 0.41161557151495554),
 ('จัดการเรียนการสอนออกเป็นหลักสูตรปริญญาตรี 13 หลักสูตร และระดับบัณฑิตศึกษา 28 หลักสูตร คือปริญญาโท 17 หลักสูตร (มีหลักสูตรนานาชาติ 5 หลักสูตร) และปริญญาเอก 11 หลักสูตร (มีหลักสูตรนานาชาติ 6 หลักสูตร)',
  0.49062960045314374),
 ('คณะวิทยาศาสตร์ มหาวิทยาลัยสงขลานครินทร์ วิทยาเขตหาดใหญ่ จัดตั้งขึ้นในปี พ.ศ. 2510 ซึ่งเป็นปีเดียวกับการก่อตั้งมหาวิทยาลัยสงขลานครินทร์',
  0.554958429797177),
 ('คณบดี คือ ศาสตราจารย์ ดร.อัญชนา ประเทพ', 0.6587029906059741)]

In [46]:
import ollama

def generate_response(conn, query_text):
    retrived_docs = query_tidb(conn, query_text)
    print(retrived_docs)
    context = "\n".join([doc[0] for doc in retrived_docs])
    print("---------")
    print(context)
    prompt = f"Answer the question based on the following context:\n{context}\n\nQuestion: {query_text}"
    #prompt
    print("---------")
    print(prompt)
    response = ollama.chat(model="llama3.2", messages=[
        {"role": "system", "content": "คุณคือประธานสาขาเทคโนโลยีสารสนเทศและการสื่อสาร มหาวิทยาลัยสงขลานครินทร์"},
        {"role": "user", "content": prompt}
    ])
    #response
    print("---------")
    print(response)
    print(response["message"]["content"])
    return response["message"]["content"]

answer = generate_response(conn, "คณะวิทย์มีกี่สาขา")
print("---------")
answer

[('คณะวิทยาศาสตร์ ประกอบด้วย 4 สาขา ได้แก่ วิทยาศาสตร์กายภาพ วิทยาศาสตร์ชีวภาพ วิทยาศาสตร์การคำนวณ และวิทยาศาสตร์สุขภาพและวิทยาศาสตร์ประยุกต์', 0.31150701789829793), ('วิสัยทัศน์ คือ คณะวิทยาศาสตร์เพื่อการพัฒนาที่ยั่งยืน', 0.4128093653717959), ('จัดการเรียนการสอนออกเป็นหลักสูตรปริญญาตรี 13 หลักสูตร และระดับบัณฑิตศึกษา 28 หลักสูตร คือปริญญาโท 17 หลักสูตร (มีหลักสูตรนานาชาติ 5 หลักสูตร) และปริญญาเอก 11 หลักสูตร (มีหลักสูตรนานาชาติ 6 หลักสูตร)', 0.45789191377802463), ('คณะวิทยาศาสตร์ มหาวิทยาลัยสงขลานครินทร์ วิทยาเขตหาดใหญ่ จัดตั้งขึ้นในปี พ.ศ. 2510 ซึ่งเป็นปีเดียวกับการก่อตั้งมหาวิทยาลัยสงขลานครินทร์', 0.5457518316950445), ('คณบดี คือ ศาสตราจารย์ ดร.อัญชนา ประเทพ', 0.6554946601390619)]
---------
คณะวิทยาศาสตร์ ประกอบด้วย 4 สาขา ได้แก่ วิทยาศาสตร์กายภาพ วิทยาศาสตร์ชีวภาพ วิทยาศาสตร์การคำนวณ และวิทยาศาสตร์สุขภาพและวิทยาศาสตร์ประยุกต์
วิสัยทัศน์ คือ คณะวิทยาศาสตร์เพื่อการพัฒนาที่ยั่งยืน
จัดการเรียนการสอนออกเป็นหลักสูตรปริญญาตรี 13 หลักสูตร และระดับบัณฑิตศึกษา 28 หลักสูตร คือปริญญาโท 17 หลัก

'คณะวิทยาศาสตร์มี 4 สาขา ได้แก่ วิทยาศาสตร์กายภาพ วิทยาศาสตร์ชีวภาพ วิทยาศาสตร์การคำนวณ และวิทยาศาสตร์สุขภาพและวิทยาศาสตร์ประยุกต์'

In [47]:
answer

'คณะวิทยาศาสตร์มี 4 สาขา ได้แก่ วิทยาศาสตร์กายภาพ วิทยาศาสตร์ชีวภาพ วิทยาศาสตร์การคำนวณ และวิทยาศาสตร์สุขภาพและวิทยาศาสตร์ประยุกต์'